Pytorch DLRC:  Implementing deep learning recommender systems

In [15]:
!pip install fbgemm-gpu
#!pip install fbgemm-gpu-cpu

!pip install torch
!pip install torchrec
#!pip install torchrec-cpu
!pip install pyre_extensions

#clear_output()


In [16]:
import pandas as pd
import numpy as np
import random

import torch
from torch.utils.data.dataset import IterableDataset
from torchrec.datasets.utils import Batch
from torchrec.sparse.jagged_tensor import KeyedJaggedTensor

## Generate Data

In [17]:
n_rows = 10000

train_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})

n_rows = int(n_rows*0.2)

test_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})

n_rows = int(n_rows*0.2)

val_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})

In [22]:


cols_dense = ["feat1", "feat2"]
cols_sparse = ['cat1', 'cat2']

In [23]:
def encode(col, map_rev):
  return [map_rev.get(item, item) for item in col]

In [24]:
map_sparse = {}
map_sparse_rev = {}

In [25]:
for feat in cols_sparse:
    map_sparse[feat] = {i: c for i,c in enumerate(train_df[feat].explode().value_counts().index)}
    map_sparse_rev[feat] = {v:k for k, v in map_sparse[feat].items()}

    train_df[feat+'_enc'] = train_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    test_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    val_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))

In [26]:
map_sparse

{'cat1': {0: 'C', 1: 'A', 2: 'B'}, 'cat2': {0: 'B', 1: 'A', 2: 'C'}}

In [28]:
train_df.head()

,feat1,feat2,cat1,cat2,label,cat1_enc,cat2_enc
0,0.296585,0.366880,A,"[A, B, C]",1,[1],"[1, 0, 2]"
1,0.445869,0.982758,A,"[A, B, C]",0,[1],"[1, 0, 2]"
2,0.156262,0.743246,B,"[B, C]",1,[2],"[0, 2]"
3,0.547944,0.011937,A,"[A, C]",0,[1],"[1, 2]"
4,0.339481,0.404834,A,"[A, B, C]",0,[1],"[1, 0, 2]"


In [29]:
test_df.head()

,feat1,feat2,cat1,cat2,label,cat1_enc,cat2_enc
0,0.490581,0.325681,A,"[B, C]",1,[1],"[0, 2]"
1,0.607796,0.566433,A,"[A, C]",1,[1],"[1, 2]"
2,0.723531,0.144988,B,"[A, B, C]",1,[2],"[1, 0, 2]"
3,0.214581,0.723466,B,"[A, B, C]",0,[2],"[1, 0, 2]"
4,0.575810,0.828940,C,"[A, C]",0,[0],"[1, 2]"


In [30]:
# Build Data batches

In [31]:

train_data = RecBatch(
    data = train_df,
    cols_sparse = [c+'_enc' for c in cols_sparse],
    cols_dense = cols_dense,
    col_label = "label",
    batch_size = 100,
    num_generated_batches = 20,
    seed = 123,
    device = device
)

NameError: name 'RecBatch' is not defined